In [12]:
import pandas as pd
import numpy as np

# File path
file_path = 'nytaxi2022.csv'

# Chunk size
chunk_size = 3e6

# Columns to check for outliers
columns_to_check = ['passenger_count', 'trip_distance', 'extra', 'total_amount']

# List to store 99th percentiles for each chunk
percentiles_list = []

# Read the dataset in chunks, only loading needed columns
for chunk in pd.read_csv(file_path, usecols=columns_to_check, chunksize=chunk_size):
    # Calculate the 99th percentile for each column in the chunk
    percentiles = chunk[columns_to_check].quantile(0.999, numeric_only=True)
    percentiles_list.append(percentiles)

# Combine all percentiles into a DataFrame
percentiles_df = pd.DataFrame(percentiles_list)

# Find the maximum 99th percentile for each column
max_99th_percentiles = percentiles_df.max()

# Display the result
print(max_99th_percentiles)

passenger_count      6.00000
trip_distance       31.47000
extra                9.75000
total_amount       169.24515
dtype: float64


In [14]:
import pandas as pd

# List to collect all rows exceeding the 99th percentile across all chunks
exceeding_rows_list = []

# Read the dataset in chunks and collect exceeding rows (only needed columns)
for chunk in pd.read_csv(file_path, usecols=columns_to_check, chunksize=chunk_size):
    # Boolean DataFrame: True where value exceeds 99th percentile
    mask = chunk[columns_to_check] > max_99th_percentiles

    # Rows where any column exceeds
    exceeding_rows = mask.any(axis=1)

    # Subset of rows exceeding
    rows_exceeding_99th = chunk[exceeding_rows].copy()

    # Add a new column listing which columns exceed
    rows_exceeding_99th['columns_exceeding'] = mask[exceeding_rows].apply(
        lambda x: list(x.index[x].values), axis=1
    )

    exceeding_rows_list.append(rows_exceeding_99th)

    print(f"Processed chunk, found {len(rows_exceeding_99th)} rows exceeding 99th percentiles.")

# Concatenate all exceeding rows into a single DataFrame
if exceeding_rows_list:
    all_exceeding_rows = pd.concat(exceeding_rows_list, ignore_index=True)
else:
    all_exceeding_rows = pd.DataFrame(columns=columns_to_check + ['columns_exceeding'])

# Display all rows that exceed the 99th percentile in any chunk
print(all_exceeding_rows)


Processed chunk, found 2562 rows exceeding 99th percentiles.
Processed chunk, found 2075 rows exceeding 99th percentiles.
Processed chunk, found 2655 rows exceeding 99th percentiles.
Processed chunk, found 3259 rows exceeding 99th percentiles.
Processed chunk, found 3090 rows exceeding 99th percentiles.
Processed chunk, found 3225 rows exceeding 99th percentiles.
Processed chunk, found 3892 rows exceeding 99th percentiles.
Processed chunk, found 3958 rows exceeding 99th percentiles.
Processed chunk, found 3887 rows exceeding 99th percentiles.
Processed chunk, found 3557 rows exceeding 99th percentiles.
Processed chunk, found 3110 rows exceeding 99th percentiles.
Processed chunk, found 2742 rows exceeding 99th percentiles.
Processed chunk, found 3623 rows exceeding 99th percentiles.
Processed chunk, found 1406 rows exceeding 99th percentiles.
       passenger_count  trip_distance  extra  total_amount  \
0                  1.0           0.00   0.00        282.80   
1                  4.0